In [1]:
import os
import pandas as pd
import numpy as np
import csv
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from string import punctuation
import nltk
from string import punctuation
from operator import itemgetter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
import string 
from collections import Counter
from nltk import word_tokenize, pos_tag, pos_tag_sents
import matplotlib.pyplot as plt

In [26]:
#This function extracts only the noun words from the reviewtext
def noun_words(df):
    df['POS'] = pos_tag_sents(df['ReviewTxt'].apply(word_tokenize).tolist())
    df['noun_words'] = np.nan
    df['noun_words_sentec'] = np.nan
    df = df.astype('object')
    for j in range(len(df)):
        tagged = df.iloc[j,9]
        noun_words = []
        for item in tagged:
            if item[1] == 'N' or item[1] == 'NN' or item[1] == 'NNP' or item[1] == 'NNS' or item[1] == 'NNPS':
                noun_words.append(item[0])
        df.iloc[j,10] = noun_words
    for j in range(len(df)):
        temp = ' '.join(word for word in df.iloc[j,10])
        df.iloc[j,11] = temp
    return df

In [25]:
def best_model_lda(df):
    no_features = 1000 #top 1000 features are selected
    tf_vectorizer = CountVectorizer(max_df=0.95,min_df=0.02,stop_words='english')
    tf = tf_vectorizer.fit_transform(df['noun_words_sentec'])
    tf_feature_names = tf_vectorizer.get_feature_names()
    search_params = {'n_components': [5, 10, 15, 20, 30], 'learning_decay': [.5, .7, .9]}

    # Init the Model
    lda = LatentDirichletAllocation()

    # Init Grid Search Class
    model = GridSearchCV(lda, param_grid=search_params)

    # Do the Grid Search
    model.fit(tf)
    # Best Model
    best_lda_model = model.best_estimator_

    # Model Parameters
    print("Best Model's Params: ", model.best_params_)

    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)

    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(tf))
    return best_lda_model

In [24]:
# Create Document - Topic Matrix
def tdf(reviews_noun):
    no_features = 1000 #top 1000 features are selected
    tf_vectorizer = CountVectorizer(max_df=0.95,min_df=0.02,stop_words='english')
    tf = tf_vectorizer.fit_transform(df['noun_words_sentec'])
    tf_feature_names = tf_vectorizer.get_feature_names()
    return (tdf, tf_vectorizer)

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=tf_vectorizer, lda_model=lda_model, n_words=100):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [ ]:
#Read the file and store it in dataframe
os.chdir("E:\\Consultancy\\Crayon")
df = DataFrame.from_csv("100K_Restaurants_reviews.tsv", sep="\t")
#extract noun words and append it to original dataframe
new_df = noun_words(df)

#identify the best model
best_model = best_model_lda(new_df)

#term document frequency matrix
tf,tf_vectorizer = tdf(new_df['noun_words_sentec'])

#Topic distribution over the review text
lda_output = best_model.transform(tf)
Topic_distribution = DataFrame(data=lda_output)
Topic_distribution.to_csv("Topic_distribution.csv")


# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_model.components_)

# Assign Column and Index
df_topic_keywords.columns = tf_vectorizer.get_feature_names()

# Show top n keywords for each topic
topic_keywords = show_topics(vectorizer=tf_vectorizer, lda_model=best_model, n_words=100)        


# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]


#Append restaurant and user id to the topic distribution from the original dataframe 
# before running the below code
Final_topics = pd.DataFrame.from_csv("Topic_distribution.csv")
Restaurant = Final_topics.groupby(['RestaurantId']).mean()
Restaurant.to_csv("Restaurant_topics_distribution.csv")

User_ID = Final_topics.groupby(['UserId']).mean()
User_ID.to_csv("User_topics_distribution.csv")